<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=b674b879d12b3e7d6fb5657db89ba0fca6c57ebfbe05a85973e355c2ddf7ba6c
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-25 11:04:29
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.42 C
-------------------
Today PnL: -60.32 K (-0.42%)
Current PnL: -23.88 L (-15.65%)
CY Booked + Current PnL: -9.99 L (-6.55%)
-------------------
Total profit:  1.22 L
Total loss:  -25.10 L
-------------------
Total Booked + Current PnL: 16.99 L (13.52%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.79%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 89.29 L (62.92%)
Deployed:  1.26 C
Current:  1.42 C
CAGR/XIRR %: 6.93%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.00,0.17,-14.07,16.48,0.09,14271.0,-14179.0,86598.0,101.07,50.0,M-SC,3.98,253.0,-0.99,0.61,13.77,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,-0.35,-9.65,10.70,0.02,16875.0,-16850.0,157709.0,140.51,69.0,M-SC,6.65,234.0,-1.00,1.11,37.18,OX40N,NTT,PAINTS
35,ICICIGI,2252.93,0.16,6.74,12.61,20.19,23463.0,11745.0,186065.0,-16.25,50.0,X-MC,7.58,68.0,0.50,1.31,22.87,X40,ATH,INSURANCE
43,ITC,452.00,-0.15,-1.55,12.17,10.44,23980.0,-3095.0,197043.0,-41.15,42.0,X-LC,0.83,5.0,-0.13,1.39,3.93,X40,NTT,FMCG
52,MEDANTA,1486.00,-0.58,3.81,19.52,24.08,25725.0,4840.0,131790.0,-4.53,47.0,X-SC,4.93,89.0,0.19,0.93,24.18,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,ROUTE,2227.21,3.58,-49.53,234.92,69.05,157784.0,-65901.0,67165.0,-59.29,37.0,H-SC,23.56,140.0,-0.42,0.47,3.58,SR,ATH,IT
61,REPCOHOME,880.00,2.13,-11.14,113.62,89.82,289259.0,-31920.0,254585.0,-55.29,49.0,H-SC,10.40,160.0,-0.11,1.79,34.69,XY24,NTT,FINANCE
64,SFL,1287.00,1.97,-40.02,111.12,26.63,174766.0,-104941.0,157277.0,19.19,32.0,M-SC,4.50,239.0,-0.60,1.11,1.97,XY24,NTT,MISC
6,ASIANTILES,137.00,1.96,-11.54,105.37,81.67,87443.0,-10823.0,82987.0,7461.11,55.0,L-SC,18.85,271.0,-0.12,0.58,60.71,XR,NTT,CERAMICS
18,COFFEEDAY,80.00,1.93,-38.30,119.24,35.27,83541.0,-43488.0,70061.0,-51.71,36.0,L-SC,17.78,270.0,-0.52,0.49,71.48,XR,NTT,HOTELS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,485.00,-5.51,-16.99,84.90,53.50,212671.0,-51255.0,250496.0,-59.80,40.0,X-MC,4.05,58.0,-0.24,1.77,11.26,XY24,NTT,FMCG
34,HONAUT,58357.33,-3.58,-14.13,66.90,43.32,93566.0,-23012.0,139860.0,-26.21,43.0,X-SC,2.01,90.0,-0.25,0.99,7.39,X40N,ATH,ELECTRICAL
28,GREENPANEL,537.00,-3.13,-31.90,110.18,43.14,137377.0,-58394.0,124684.0,179.07,30.0,M-SC,5.54,240.0,-0.43,0.88,17.28,XY24,NTT,MISC
3,ACC,3906.00,-2.28,-21.41,108.87,64.15,203598.0,-50941.0,187010.0,-54.13,54.0,X-SC,1.63,82.0,-0.25,1.32,4.90,XY24,BTT,CEMENT
57,RAJESHEXPO,518.00,-2.18,-63.83,176.64,0.07,87651.0,-87556.0,49621.0,1718.18,53.0,L-SC,1.53,268.0,-1.00,0.35,23.41,OX40N,NTT,JEWELLERY


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,0.37,-0.48,107.71,106.72,132212.0,-588.0,122748.0,-50.63,34.0,H-SC,3.71,139.0,-0.00,0.86,22.79,AR,ATH,MISC
85,WIPRO,420.00,-0.68,0.73,71.02,72.27,121216.0,1230.0,170678.0,-13.34,53.0,M-LC,4.55,99.0,0.01,1.20,6.95,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.17,-5.44,119.87,107.91,167355.0,-8030.0,139614.0,-22.98,32.0,M-SC,10.49,216.0,-0.05,0.98,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.35,-9.65,10.70,0.02,16875.0,-16850.0,157709.0,140.51,69.0,M-SC,6.65,234.0,-1.00,1.11,37.18,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.17,-14.07,16.48,0.09,14271.0,-14179.0,86598.0,101.07,50.0,M-SC,3.98,253.0,-0.99,0.61,13.77,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.74,-23.15,47.65,13.47,98751.0,-62424.0,207243.0,-68.31,25.0,H-SC,1.38,158.0,-0.63,1.46,5.38,XY24,NTT,PAINTS
44,JCHAC,2282.00,-0.40,-23.51,30.72,-0.01,30032.0,-30046.0,97759.0,19900.00,54.0,M-SC,1.67,233.0,-1.00,0.69,10.70,OX40N,NTT,AC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.0,-0.68,0.73,71.02,72.27,121216.0,1230.0,170678.0,-13.34,53.0,M-LC,4.55,99.0,0.01,1.2,6.95,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,-0.68,0.73,71.02,72.27,121216.0,1230.0,170678.0,-13.34,53.0,M-LC,4.55,99.0,0.01,1.20,6.95,XR,NTT,IT
38,INDIAMART,4810.62,0.37,-0.48,107.71,106.72,132212.0,-588.0,122748.0,-50.63,34.0,H-SC,3.71,139.0,-0.00,0.86,22.79,AR,ATH,MISC
86,ZYDUSLIFE,1286.17,0.58,-1.72,38.00,35.63,78269.0,-3602.0,205972.0,-15.41,37.0,H-MC,4.39,119.0,-0.05,1.45,14.67,AR,ATH,PHARMA
25,FINCABLES,1641.55,-0.17,-5.44,119.87,107.91,167355.0,-8030.0,139614.0,-22.98,32.0,M-SC,10.49,216.0,-0.05,0.98,0.00,OX40N,ATH,CABLES
84,WHIRLPOOL,2270.00,0.08,-5.11,91.72,81.92,99909.0,-5870.0,108928.0,-44.42,32.0,M-SC,4.53,236.0,-0.06,0.77,28.95,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.0,-0.91,-15.86,68.99,42.18,160442.0,-43852.0,232558.0,-24.30,21.0,X-LC,0.61,3.0,-0.27,1.64,0.00,XY24,NTT,AUTO
46,JSWINFRA,342.0,-0.43,-9.76,27.68,15.23,50045.0,-19548.0,180799.0,-22.76,23.0,X-MC,6.66,66.0,-0.39,1.27,19.35,X40N,NTT,REALTY
11,BATAINDIA,2096.0,-0.55,-35.48,109.56,35.20,90953.0,-45653.0,83017.0,8.88,24.0,X-SC,12.89,92.0,-0.50,0.59,0.00,X40,NTT,FOOTWEAR
60,RELAXO,1176.0,-0.14,-47.89,192.28,52.31,145444.0,-69518.0,75642.0,-44.54,27.0,X-SC,3.95,91.0,-0.48,0.53,1.25,X40N,NTT,FOOTWEAR
27,GLAXO,3466.2,-1.76,-2.31,42.44,39.15,82622.0,-4604.0,194680.0,-21.47,28.0,X-MC,9.50,60.0,-0.06,1.37,26.02,X40,ATH,PHARMA


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.0,-0.91,-15.86,68.99,42.18,160442.0,-43852.0,232558.0,-24.30,21.0,X-LC,0.61,3.0,-0.27,1.64,0.00,XY24,NTT,AUTO
43,ITC,452.0,-0.15,-1.55,12.17,10.44,23980.0,-3095.0,197043.0,-41.15,42.0,X-LC,0.83,5.0,-0.13,1.39,3.93,X40,NTT,FMCG
36,ICICIPRULI,790.0,0.61,1.30,29.29,30.97,52440.0,2300.0,179038.0,-22.52,49.0,X-MC,1.57,75.0,0.04,1.26,14.03,X40,ATH,INSURANCE
3,ACC,3906.0,-2.28,-21.41,108.87,64.15,203598.0,-50941.0,187010.0,-54.13,54.0,X-SC,1.63,82.0,-0.25,1.32,4.90,XY24,BTT,CEMENT
66,SIEMENS,4671.5,-0.08,-15.06,47.76,25.51,75497.0,-28020.0,158075.0,-2.87,52.0,H-LC,1.86,49.0,-0.37,1.11,16.49,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.55,-35.48,109.56,35.20,90953.0,-45653.0,83017.0,8.88,24.0,X-SC,12.89,92.0,-0.50,0.59,0.00,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,-0.29,-13.75,72.12,48.45,124723.0,-27568.0,172938.0,-19.87,30.0,X-MC,6.82,64.0,-0.22,1.22,0.00,X40N,ATH,FINANCE
76,TMPV,600.00,-0.91,-15.86,68.99,42.18,160442.0,-43852.0,232558.0,-24.30,21.0,X-LC,0.61,3.0,-0.27,1.64,0.00,XY24,NTT,AUTO
54,PAGEIND,51605.08,-0.31,-5.36,33.21,26.07,51462.0,-8780.0,154960.0,-30.31,34.0,X-MC,9.03,55.0,-0.17,1.09,0.45,X40,ATH,APPARELS
55,PGHH,17907.65,-0.40,-4.50,40.07,33.76,76844.0,-9045.0,191775.0,-32.64,33.0,X-MC,3.36,57.0,-0.12,1.35,0.46,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,1.61,-27.99,98.36,42.84,46043.0,-18195.0,46811.0,-52.62,40.0,X-SC,37.37,83.0,-0.40,0.33,1.86,XY24,NTT,MISC
60,RELAXO,1176.00,-0.14,-47.89,192.28,52.31,145444.0,-69518.0,75642.0,-44.54,27.0,X-SC,3.95,91.0,-0.48,0.53,1.25,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.55,-35.48,109.56,35.20,90953.0,-45653.0,83017.0,8.88,24.0,X-SC,12.89,92.0,-0.50,0.59,0.00,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-1.98,-5.79,29.06,21.58,38365.0,-8118.0,132021.0,-52.49,31.0,X-MC,7.05,56.0,-0.21,0.93,16.88,X40N,ATH,IT
52,MEDANTA,1486.00,-0.58,3.81,19.52,24.08,25725.0,4840.0,131790.0,-4.53,47.0,X-SC,4.93,89.0,0.19,0.93,24.18,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,-0.54,-15.11,40.52,19.28,118993.0,-52291.0,293665.0,-25.89,58.0,X-LC,7.53,1.0,-0.44,2.07,8.56,X40,ATH,IT
41,INFY,2275.00,-1.15,4.17,48.67,54.87,158632.0,13045.0,325933.0,-18.88,56.0,X-LC,3.01,2.0,0.08,2.30,12.80,X40,BTT,IT
76,TMPV,600.00,-0.91,-15.86,68.99,42.18,160442.0,-43852.0,232558.0,-24.30,21.0,X-LC,0.61,3.0,-0.27,1.64,0.00,XY24,NTT,AUTO
82,VBL,671.64,0.24,-9.68,50.20,35.67,143212.0,-30560.0,285282.0,-20.46,41.0,X-LC,5.95,4.0,-0.21,2.01,2.84,X40N,ATH,FMCG
43,ITC,452.00,-0.15,-1.55,12.17,10.44,23980.0,-3095.0,197043.0,-41.15,42.0,X-LC,0.83,5.0,-0.13,1.39,3.93,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,1.96,-11.54,105.37,81.67,87443.0,-10823.0,82987.0,7461.11,55.0,L-SC,18.85,271.0,-0.12,0.58,60.71,XR,NTT,CERAMICS
68,SONACOMS,806.63,-1.13,-13.48,61.29,39.54,53640.0,-13641.0,87518.0,-32.36,62.0,M-SC,6.69,220.0,-0.25,0.62,23.41,AR,ATH,AUTO
51,MASFIN,398.61,-0.08,-5.28,28.85,22.05,26774.0,-5175.0,92805.0,-18.25,50.0,H-SC,7.33,164.0,-0.19,0.65,34.90,XR,ATH,FINANCE
70,SURYODAY,216.00,-0.14,-19.94,53.68,23.03,76956.0,-35710.0,143361.0,55.70,48.0,H-SC,10.59,167.0,-0.46,1.01,42.24,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,-0.35,-9.65,10.70,0.02,16875.0,-16850.0,157709.0,140.51,69.0,M-SC,6.65,234.0,-1.00,1.11,37.18,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-0.35,-9.65,10.70,0.02,16875.0,-16850.0,157709.0,140.51,69.0,M-SC,6.65,234.0,-1.00,1.11,37.18,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.26,0.54,1.18,19.54,20.95,49775.0,2966.0,254734.0,0.21,77.0,X-LC,13.41,31.0,0.06,1.80,37.23,X40,ATH,PAINTS
40,INDUSINDBK,1800.00,-0.62,-38.66,116.63,32.89,55237.0,-29846.0,47361.0,-684.76,57.0,L-MC,5.55,259.0,-0.54,0.33,30.43,XR,NTT,BANKS
12,BERGEPAINT,680.00,-0.60,2.59,18.09,21.14,42189.0,5880.0,233219.0,-4.59,61.0,X-MC,3.89,74.0,0.14,1.64,31.45,XY24,NTT,PAINTS
79,UNITDSPR,1644.00,-0.11,5.02,15.00,20.77,36884.0,11747.0,245891.0,-4.14,59.0,X-MC,3.56,62.0,0.32,1.73,12.28,X40N,NTT,BREWERIES


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.71
1,25,44.60
2,50,75.50


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.64
MC    29.22
LC    25.13
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.96
X40      21.21
X40N     11.89
XR        9.61
XY25      9.08
AR        8.96
OX40N     7.66
X200      1.85
SR        0.99
MH        0.78
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.89
X-MC    22.78
X-LC    20.00
M-SC    12.07
X-SC     8.26
H-MC     4.73
H-LC     2.96
L-SC     1.42
M-MC     1.38
M-LC     1.20
L-LC     0.97
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.75,-5.98,40.26
IT,13.73,-16.00,75.88
FINANCE,9.46,-15.37,64.12
MISC,7.43,-23.30,73.81
PAINTS,6.01,-8.26,24.34
ELECTRICAL,5.91,-12.00,53.03
INSURANCE,4.44,-0.17,35.47
PHARMA,3.86,-1.85,34.45
AUTO,3.34,-21.38,73.42


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3069648.0,21
AR,1300047.0,10
XR,1294243.0,13
X40,1030439.0,14
X40N,761675.0,9
OX40N,704444.0,10
XY25,354145.0,6
SR,279426.0,2
MH,73976.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3584439.0,25
M-SC,1346077.0,15
X-MC,1248791.0,16
X-LC,927810.0,11
X-SC,750253.0,8
H-MC,408324.0,3
L-SC,258635.0,3
H-LC,136503.0,2
M-LC,121216.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1222011.0      6
           AR         882463.0      5
           XR         797104.0      7
M-SC       XY24       784088.0      6
X-MC       X40        451340.0      7
X-LC       X40        402634.0      5
X-MC       XY24       363096.0      3
H-SC       OX40N      329459.0      4
X-SC       X40N       298422.0      3
M-SC       OX40N      287334.0      5
H-SC       SR         279426.0      2
X-SC       XY24       275366.0      3
X-MC       X40N       250017.0      4
X-LC       XY24       233897.0      2
H-MC       AR         217134.0      2
X-LC       X40N       213236.0      2
H-MC       XY24       191190.0      1
X-MC       XY25       184338.0      2
X-SC       X40        176465.0      2
L-SC       XR         170984.0      2
M-SC       XR         149702.0      2
           AR         124953.0      2
M-LC       XR         121216.0      1
L-SC       OX40N       87651.0      1
X-LC       XY25        78043.0      2
H-LC       AR          75497.0      1
H-SC       MH          73976.0      1
H-LC       X200        61006.0      1
L-MC       XR          55237.0      1
M-MC       XY25        49922.0      1
L-LC       XY25        41842.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 25.0 seconds
